In [1]:
from automatic_variable_mapping import corpus, vocab_similarity
import pandas as pd
from functools import partial
import time
import numpy as np
from automatic_variable_mapping.vocab_similarity import default_pairable
from gensim.models import KeyedVectors

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Obs Heart Studies
#input_data_type = "obs_heart_studies_dbGaP_NLP"
#ref_id_col = 'dbGaP_studyID_datasetID_varID'
#doc_cols_inputs = {
#     'desc': ['variable_description'],
#     'units': ['units'],
#     'coding': ['variable_coding_counts_distribution'],
#     #'desc_units': ['variable_description', 'units'],
#     'desc_coding': ['variable_description', 'units', 'variable_coding_counts_distribution'],
#     'desc_units_coding': ['variable_description', 'units', 'variable_coding_counts_distribution']
# }
# mult_doc_cols_inputs = {"desc": ['variable_description']}
# input_dir = "/Laura/tiff_laura_shared/"
# ref_file_diff = ""

In [3]:
# # HF Clin Trials
# input_data_type = "HF_clin_trials_biolincc_NLP"
# ref_id_col = 'var_doc_id'
# doc_cols_inputs = {'desc': ['variable_description']}
# mult_doc_cols_inputs = {"desc": ['variable_description']}
# input_dir = "/Laura/tiff_laura_shared/"
# ref_file_diff = ""

In [4]:
# Both HF Clin Trials and Obs Heart Studies
#input_data_type = "trials_studies_NLP"
#ref_id_col = 'var_doc_id'
#doc_cols_inputs = {'desc': ['variable_description']}
#mult_doc_cols_inputs = {"desc": ['variable_description']}
#input_dir = "/Laura/tiff_laura_shared/"
#ref_file_diff = ""

In [32]:
# Both SNOMED and Obs Heart Studies
input_data_type = "obs_heart_studies_dbGaP_NLP_SNOMED_terms"
ref_id_col = 'dbGaP_studyID_datasetID_varID'
doc_cols_inputs = {'desc': ['variable_description']}
mult_doc_cols_inputs = {"desc": ['variable_description']}
input_dir = "/Laura/automatic-variable-mapping/SNOMED-concepts/output/"
man_file = input_dir + "manual_concept_var_mappings_" + "obs_heart_studies_dbGaP_NLP" + ".csv"

In [33]:
data_file = input_dir + "var_doc_" + input_data_type + ".csv"
man_file = input_dir + "manual_concept_var_mappings_" + input_data_type + ".csv"

In [34]:
output_dir = "/data/automatic-variable-mapping-results-2/"
out_prefix = "var_txt_sim_scores"
num_cpus = 14

In [44]:
print data_file
print man_file

/Laura/automatic-variable-mapping/SNOMED-concepts/output/var_doc_obs_heart_studies_dbGaP_NLP_SNOMED_terms.csv
/Laura/tiff_laura_shared/manual_concept_var_mappings_obs_heart_studies_dbGaP_NLP.csv


In [45]:
data = pd.read_csv(man_file,
                       sep=",",
                       quotechar='"',
                       na_values="",
                       low_memory=False)
data.shape
#data[data["var_doc_id"] == "escape.admreas.adothr"]
data

,study,cohort_dbGaP,dbGaP_studyID_datasetID,dbGaP_dataset_label,concept,data_desc,varID,var_desc,units,var_coding_counts_distribution,var_doc_id,concept_id,dbGaP_studyID_datasetID_varID,var_coding_labels
0,FHS,Original,phs000007.v26.pht000009.v2,ex0_7s,Education,"Clinic Exam, Original Cohort Exams 1 - 7",MF5,EDUCATION,NaN,n=4884; nulls=195; HIGH SCHOOL GRADUATE=5:coun...,FHS1var2,education,phs000007.v26.pht000009.v2.MF5,"FIFTH, SIXTH OR SEVENTH GRADE; NONE; POST GRAD..."
1,FHS,Original,phs000007.v26.pht000009.v2,ex0_7s,number of pregnancies,"Clinic Exam, Original Cohort Exams 1 - 7",MF20,"NUMBER OF PREGNANCIES, EXAM 1",NUMBER OF PREGNANCIES,n=2122; nulls=17; mean=2.82; sd=1.353; median=...,FHS1var16,pregnancies,phs000007.v26.pht000009.v2.MF20,"MAN, SINGLE WOMAN, OR NO PREGNANCIES FOR NON-S..."
2,FHS,Original,phs000007.v26.pht000009.v2,ex0_7s,unique Participant Identifier,"Clinic Exam, Original Cohort Exams 1 - 7",shareid,UNIQUE PARTICIPANT ID,NaN,n=5079; nulls=0,FHS1var558,ParticipantID,phs000007.v26.pht000009.v2.shareid,NaN
3,FHS,Original,phs000007.v26.pht000010.v3,ex0_8s,unique Participant Identifier,"Clinic Exam, Original Cohort Exam 8",shareid,UNIQUE PARTICIPANT ID,NaN,n=3909; nulls=0,FHS2var151,ParticipantID,phs000007.v26.pht000010.v3.shareid,NaN
4,FHS,Original,phs000007.v26.pht000011.v3,ex0_9s,Marital Status,"Clinic Exam, Original Cohort Exam 9",FB8,MARTIAL STATUS,NaN,n=3712; nulls=0; MARRIED=2:count=2847; WIDOWED...,FHS3var1,maritalStatus,phs000007.v26.pht000011.v3.FB8,MARRIED; SEPARATED; SINGLE; WIDOWED; DIVORCED
5,FHS,Original,phs000007.v26.pht000011.v3,ex0_9s,Systolic Blood Pressure (mmHg),"Clinic Exam, Original Cohort Exam 9",FB18,"BLOOD PRESSURE: SYSTOLIC, TAKEN BY NURSE",MM HG,n=3698; nulls=14; mean=138.7; sd=22.16; median...,FHS3var8,SBP,phs000007.v26.pht000011.v3.FB18,NaN
6,FHS,Original,phs000007.v26.pht000011.v3,ex0_9s,Diastolic Blood Pressure (mmHg),"Clinic Exam, Original Cohort Exam 9",FB19,"BLOOD PRESSURE: DIASTOLIC, TAKEN BY NURSE",MM HG,n=3699; nulls=13; mean=85.64; sd=11.88; median...,FHS3var9,DBP,phs000007.v26.pht000011.v3.FB19,NaN
7,FHS,Original,phs000007.v26.pht000011.v3,ex0_9s,Systolic Blood Pressure (mmHg),"Clinic Exam, Original Cohort Exam 9",FB20,"BLOOD PRESSURE: SYSTOLIC, PHYSICIAN'S FIRST RE...",MM HG,n=3708; nulls=4; mean=140.1; sd=23.46; median=...,FHS3var10,SBP,phs000007.v26.pht000011.v3.FB20,NaN
8,FHS,Original,phs000007.v26.pht000011.v3,ex0_9s,Diastolic Blood Pressure (mmHg),"Clinic Exam, Original Cohort Exam 9",FB21,"BLOOD PRESSURE: DIASTOLIC, PHYSICIAN'S FIRST R...",MM HG,n=3705; nulls=7; mean=82.19; sd=11.59; median=...,FHS3var11,DBP,phs000007.v26.pht000011.v3.FB21,NaN
9,FHS,Original,phs000007.v26.pht000011.v3,ex0_9s,Systolic Blood Pressure (mmHg),"Clinic Exam, Original Cohort Exam 9",FB22,"BLOOD PRESSURE: SYSTOLIC, PHYSICIAN'S SECOND R...",MM HG,n=3651; nulls=61; mean=139.1; sd=23.49; median...,FHS3var12,SBP,phs000007.v26.pht000011.v3.FB22,NaN


In [29]:
data2

,dbGaP_study_id,dbGaP_dataset_id,dataset_description,variable_id,variable_description,units,variable_coding_labels,study,dbGaP_dataset_label,variable_coding_counts_distribution,var_doc_id,study_dataset_dbGaP_id,dbGaP_studyID_datasetID_varID
0,phs000007.v26,pht000009.v2,"Clinic Exam, Original Cohort Exams 1 - 7",MF4,"RELATIVE WEIGHT, EXAM 1",PERCENT,NaN,FHS,ex0_7s,n=5036; nulls=43; mean=102.2; sd=16.8; median=...,FHS1var1,phs000007.v26.pht000009.v2,phs000007.v26.pht000009.v2.MF4
1,phs000007.v26,pht000009.v2,"Clinic Exam, Original Cohort Exams 1 - 7",MF5,EDUCATION,NaN,"FIFTH, SIXTH OR SEVENTH GRADE; NONE; POST GRAD...",FHS,ex0_7s,n=4884; nulls=195; HIGH SCHOOL GRADUATE=5:coun...,FHS1var2,phs000007.v26.pht000009.v2,phs000007.v26.pht000009.v2.MF5
2,phs000007.v26,pht000009.v2,"Clinic Exam, Original Cohort Exams 1 - 7",MF6,COUNTRY OF BIRTH,NaN,BORN IN UNITED STATES; NOT BORN IN UNITED STATES,FHS,ex0_7s,n=5079; nulls=0; BORN IN UNITED STATES=0:count...,FHS1var3,phs000007.v26.pht000009.v2,phs000007.v26.pht000009.v2.MF6
3,phs000007.v26,pht000009.v2,"Clinic Exam, Original Cohort Exams 1 - 7",MF8,"HISTORY OF ACUTE INFECTIONS, EXAM 1",NaN,SCARLET FEVER; NEGATIVE; DIPHTHERIA AND FREQUE...,FHS,ex0_7s,n=5029; nulls=50; NEGATIVE=0:count=3087; FREQU...,FHS1var4,phs000007.v26.pht000009.v2,phs000007.v26.pht000009.v2.MF8
4,phs000007.v26,pht000009.v2,"Clinic Exam, Original Cohort Exams 1 - 7",MF9,"HISTORY OF RHEUMATIC FEVER, EXAM 1",NaN,NONE; DOUBTFUL; UNKNOWN; YES,FHS,ex0_7s,n=5065; nulls=14; NONE=0:count=4738; YES=1:cou...,FHS1var5,phs000007.v26.pht000009.v2,phs000007.v26.pht000009.v2.MF9
5,phs000007.v26,pht000009.v2,"Clinic Exam, Original Cohort Exams 1 - 7",MF10,"HISTORY OF ALLERGY OR ASTHMA, EXAM 1",NaN,"BRONCHIAL ASTHMA, ALONE; NEGATIVE; UNKNOWN; AL...",FHS,ex0_7s,n=5019; nulls=60; NEGATIVE=0:count=4467; ALLER...,FHS1var6,phs000007.v26.pht000009.v2,phs000007.v26.pht000009.v2.MF10
6,phs000007.v26,pht000009.v2,"Clinic Exam, Original Cohort Exams 1 - 7",MF11,"HISTORY OF CHRONIC ARTHRITIS AND RHEUMATISM, E...",NaN,NO; UNKNOWN; YES,FHS,ex0_7s,n=5019; nulls=60; NO=0:count=4360; YES=1:count...,FHS1var7,phs000007.v26.pht000009.v2,phs000007.v26.pht000009.v2.MF11
7,phs000007.v26,pht000009.v2,"Clinic Exam, Original Cohort Exams 1 - 7",MF12,"HISTORY OF THYROID DISEASE, EXAM 1",NaN,HYPOTHYROID ONLY; NEGATIVE; DOUBTFUL; HYPERTHY...,FHS,ex0_7s,n=5054; nulls=25; NEGATIVE=0:count=4726; OTHER...,FHS1var8,phs000007.v26.pht000009.v2,phs000007.v26.pht000009.v2.MF12
8,phs000007.v26,pht000009.v2,"Clinic Exam, Original Cohort Exams 1 - 7",MF13,"HISTORY OF HYPERTENSION, EXAM 1",NaN,PERMANENT; NEGATIVE; DOUBTFUL; UNKNOWN; TRANSI...,FHS,ex0_7s,n=5074; nulls=5; NEGATIVE=0:count=4328; TYPE U...,FHS1var9,phs000007.v26.pht000009.v2,phs000007.v26.pht000009.v2.MF13
9,phs000007.v26,pht000009.v2,"Clinic Exam, Original Cohort Exams 1 - 7",MF14,"HISTORY OF ENLARGED HEART, EXAM 1",NaN,NO; UNKNOWN; YES,FHS,ex0_7s,n=4943; nulls=136; NO=0:count=4861; YES=1:coun...,FHS1var10,phs000007.v26.pht000009.v2,phs000007.v26.pht000009.v2.MF14


In [30]:
data2 = pd.read_csv(data_file,
                       sep=",",
                       quotechar='"',
                       na_values="",
                       low_memory=False)
data2.shape

(105611, 13)

In [78]:
pair_ids = list(data2[data2["dbGaP_study_id"]=="SNOMED"][ref_id_col])
len(pair_ids)

27946

In [11]:
# bioWordVec embeddings with regular tfidf
vec_file_dir = "/data/WordVectors2/"
binary_file_name = vec_file_dir + "BioWordVec_PubMed_MIMICIII_d200.vec.bin"
biowordvec_embeddings = KeyedVectors.load_word2vec_format(binary_file_name, binary=True, limit=None)

In [12]:
def matching_groups(data, group_col, corpora, pair_id, ref_id):
    corpus_doc_ids = [doc_id for doc_id, _ in corpora]
    ref_idx = corpus_doc_ids.index(ref_id)
    pair_idx = corpus_doc_ids.index(pair_id)
    return data[pair_idx][group_col] == data[ref_idx][group_col]


def pairable_by_group(data, group_col, corpus_doc_ids, score, pair_id, _, ref_id):
    return vocab_similarity.default_pairable(score, pair_id, None, ref_id) and not matching_groups(data, group_col, corpus_doc_ids, pair_id, ref_id)


def calc_score_results(data_file, doc_cols, doc_col_key, ref_id_col, filter_file, 
                       word_vectors=None, 
                       corpora_col=None, 
                       pair_ids_col=None):
    print data_file
    print "doc_col_key: " + doc_col_key
    data = pd.read_csv(data_file, sep=",",quotechar='"', na_values="",low_memory=False)
    
    if corpora_col:
        corpora_data = corpus.partition(data, corpora_col)
    else:
        corpora_data = [data]
    
    if filter_file != data_file:
        filter_data = pd.read_csv(filter_file,
                                  sep=",",
                                  quotechar='"',
                                  na_values="",
                                  low_memory=False)
    else:
        filter_data = data
        
    print(" Building Corpora...")
    corpora = corpus.build_corpora(doc_cols, corpora_data, ref_id_col, num_cpus=num_cpus)
    print("Corpus: ", len(corpora), " ", len([doc for c in corpora for doc in c]))
    
    print("  Calculating BOW matrix...")
    vocab, bow = corpus.calc_tfidf(corpora)
    print("tfidf matrix size: " + str(bow.shape))
    
    # calculate embeddings
    if word_vectors:
        print("  Calculating doc embeddings matrix...")
        doc_ids, doc_vectors = corpus.calc_doc_embeddings(bow.toarray(), vocab, word_vectors, corpora)
        print("doc_ids: ", len(doc_ids), " vecs: ", len(doc_vectors))
    else:
        doc_ids = [doc for c in corpora for doc, _ in c]
        doc_vectors = bow.toarray()
    
    print("doc vectors size " + str(doc_vectors.shape))
    
    tfidf_label = "multtfcdf" if corpora_col else "tfidf"
    embed_label = "embed" if word_vectors else None
    
    out_suffix = None
    
    out_file_name = str.join('_', [i for i in [out_prefix, 
                                               input_data_type, 
                                               tfidf_label, 
                                               embed_label, 
                                               doc_col_key,
                                               out_suffix] 
                                   if i])
    out_file = output_dir + out_file_name + ".csv"
    
    ref_ids = filter_data[ref_id_col]
    
    print("  Calculating Similarity Scores...")
    scores = vocab_similarity.VariableSimilarityCalculator(ref_ids, pairable=default_pairable)
    scores.init_cache(file_name=out_file)

    result = scores.score_variables(doc_ids, bow, num_cpus=num_cpus, file_name=out_file, pair_ids=pair_ids)
    
    return result

def calc_scores_doc_cols(data_file, doc_cols_inputs, ref_id_col, filter_file, corpora_col=None, word_vectors=None, pair_ids_col=None):
    [calc_score_results(data_file, 
                        doc_cols_inputs[key], 
                        key, 
                        ref_id_col, 
                        filter_file, 
                        corpora_col=corpora_col, 
                        word_vectors=word_vectors, 
                        pair_ids_col=pair_ids_col) 
     for key in doc_cols_inputs]

In [96]:
reload(corpus)
reload(vocab_similarity)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


<module 'automatic_variable_mapping.vocab_similarity' from 'automatic_variable_mapping/vocab_similarity.pyc'>

In [15]:
scores_embed = calc_scores_doc_cols(data_file, doc_cols_inputs, ref_id_col, man_file, word_vectors=biowordvec_embeddings, pair_ids_col=None)
scores_tfidf = calc_scores_doc_cols(data_file, doc_cols_inputs, ref_id_col, man_file, word_vectors=None, pair_ids_col=None)
scores_tfmcdf = calc_scores_doc_cols(data_file, mult_doc_cols_inputs, ref_id_col, man_file, word_vectors=None, corpora_col='study', pair_ids_col=None)

/Laura/tiff_laura_shared/var_doc_HF_clin_trials_biolincc_NLP.csv
doc_col_key: desc
 Building Corpora...


100%|██████████| 7028/7028 [00:16<00:00, 415.50it/s]  


('Corpus: ', 1, ' ', 7028)
  Calculating BOW matrix...
('vocab size: ', 2334)
tfidf matrix size: (7028, 2334)
  Calculating doc embeddings matrix...
29 documents deleted because document embeddings was all zeros
('doc_ids: ', 6999, ' vecs: ', 6999)
doc vectors size (6999, 200)
  Calculating Similarity Scores...
Finding valid pair ids
Pair ids: 6999
Finding valid ref ids
Ref ids: 453
Multiplying matrices
LHS: (6999, 2334)
RHS: (453, 2334)
Sim Matrix: (453, 6999)
Chunk size: 33
Finding matches for 0 carress.a_base.age
Finding matches for 100 escape.mhist.htn
Finding matches for 200 exact.a_visitsumm.cl_ntpro
Finding matches for 400 topcat.t004.ef
Finding matches for 300 relax.medhist1.alcoholc


100%|██████████| 453/453 [00:00<00:00, 329979.11it/s]


/Laura/tiff_laura_shared/var_doc_HF_clin_trials_biolincc_NLP.csv
doc_col_key: desc
 Building Corpora...


...

100%|██████████| 453/453 [00:00<00:00, 224032.51it/s]



/Laura/tiff_laura_shared/var_doc_HF_clin_trials_biolincc_NLP.csv
doc_col_key: desc
 Building Corpora...


100%|██████████| 1126/1126 [00:03<00:00, 326.51it/s]


('Corpus: ', 6, ' ', 7028)
  Calculating BOW matrix...
('vocab size: ', 2334)
tfidf matrix size: (7028, 2334)
doc vectors size (7028, 2334)
  Calculating Similarity Scores...
Finding valid pair ids
Pair ids: 7028
Finding valid ref ids
Ref ids: 453
Multiplying matrices
LHS: (7028, 2334)
RHS: (453, 2334)
Sim Matrix: (453, 7028)
Chunk size: 33
Finding matches for 0 carress.a_base.age
Finding matches for 100 escape.mhist.htn
Finding matches for 200 exact.a_visitsumm.cl_ntpro
Finding matches for 300 relax.medhist1.alcoholc
Finding matches for 400 topcat.t004.ef


100%|██████████| 453/453 [00:00<00:00, 289108.29it/s]


## Top N words from concept vectors

In [97]:
input_dir = "/Laura/tiff_laura_shared/"
top_words_prefix = "top_n_words"

In [98]:
filter_ref_id_col = ref_id_col
out_prefix = "concept_vec_sim_scores_gold_standard"
man_file_name = "manual_concept_var_mappings_obs_heart_studies_dbGaP_NLP"
man_file = input_dir + man_file_name + ".csv"
man_file_name = "concept_groups_gold_standard"
concept_id_col = "concept"

In [103]:
out_prefix = "concept_vec_sim_scores"
man_file_name = "concept_groups_reference_MESA_vars"
man_file = "/Laura/tiff_laura_shared/" + "" + man_file_name + ".csv"
concept_id_col = "pred_concept_id"
filter_ref_id_col = ref_id_col

In [106]:
out_prefix = "concept_vec_sim_scores"
man_file_name = "concept_groups_communities"
man_file = "/Laura/tiff_laura_shared/" + man_file_name + ".csv"
concept_id_col = "pred_concept_id"
filter_ref_id_col = ref_id_col

In [107]:
from scipy.sparse import csr_matrix

def csr_vappend(a,b):
    """ Takes in 2 csr_matrices and appends the second one to the bottom of the first one. 
    Much faster than scipy.sparse.vstack but assumes the type to be csr and overwrites
    the first matrix instead of copying it. The data, indices, and indptr still get copied."""

    a.data = np.hstack((a.data,b.data))
    a.indices = np.hstack((a.indices,b.indices))
    a.indptr = np.hstack((a.indptr,(b.indptr + a.nnz)[1:]))
    a._shape = (a.shape[0]+b.shape[0],b.shape[1])
    return a

def write_top_words(tops, file_name):
    with open(file_name, "w") as f:
        f.write(",".join(["Concept", "Top 5 words"]))
        f.write("\n")
        for group, words in tops.items():
            f.write(",".join([group, " ".join(words)]))
            f.write("\n")

def calc_top_n_words(data_file, 
                     doc_cols, 
                     doc_col_key,
                     ref_id_col, 
                     filter_file,
                     n,
                     concept_id_col,
                     filter_ref_id_col, 
                     word_vectors=None, 
                     corpora_col=None, 
                     pair_ids=None):
    print data_file
    print "doc_col_key: " + doc_col_key
    data = pd.read_csv(data_file, sep=",",quotechar='"', na_values="",low_memory=False)
    
    if corpora_col:
        corpora_data = corpus.partition(data, corpora_col)
    else:
        corpora_data = [data]
    
    if filter_file != data_file:
        filter_data = pd.read_csv(filter_file,
                                  sep=",",
                                  quotechar='"',
                                  na_values="",
                                  low_memory=False)
    else:
        filter_data = data
        
    print(" Building Corpora...")
    corpora = corpus.build_corpora(doc_cols, corpora_data, ref_id_col, num_cpus=num_cpus)
    print("Corpus: ", len(corpora), " ", len([doc for c in corpora for doc in c]))
    
    print("  Calculating BOW matrix...")
    vocab, bow = corpus.calc_tfidf(corpora)
    
    # calculate embeddings
    if word_vectors:
        print("  Calculating doc embeddings matrix...")
        doc_ids, doc_vectors = corpus.calc_doc_embeddings(bow.toarray(), vocab, word_vectors, corpora)
        print("doc_ids: ", len(doc_ids), " vecs: ", len(doc_vectors))
    else:
        doc_ids = [doc for c in corpora for doc, _ in c]
        doc_vectors = bow.toarray()
    
    
    tfidf_label = "multtfcdf" if corpora_col else "tfidf"
    embed_label = "embed" if word_vectors else None
    
    input_data_type = man_file_name
    out_file_name = str.join('_', [i for i in [out_prefix, 
                                               input_data_type, 
                                               tfidf_label, 
                                               embed_label, 
                                               doc_col_key] 
                                   if i])
    out_file = output_dir + out_file_name + ".csv"
    
    print("  Calculating Top N words..")
    filter_groups = filter_data.groupby([concept_id_col])[filter_ref_id_col]
    groups = {group: data[ref_id_col].isin(filter_groups.get_group(group)) for group in list(filter_groups.groups)}
    normalzed_doc_vectors_by_group = corpus.normalize_doc_vectors_by_group(doc_vectors, groups)
    top_words_in_group = {group: corpus.vec_top_words(vocab, vec, n) for group, vec in normalzed_doc_vectors_by_group.items()}

    top_words_out_file_name = str.join('_', [i for i in [top_words_prefix, 
                                                        man_file_name]
                                             if i])
    top_words_out_file = output_dir + top_words_out_file_name + ".csv"
    write_top_words(top_words_in_group, top_words_out_file)
    
    ref_ids = normalzed_doc_vectors_by_group.keys() 
    
    print("  Calculating Similarity Scores...")
    scores = vocab_similarity.VariableSimilarityCalculator(ref_ids, pairable=default_pairable)
    scores.init_cache(file_name=out_file)

    normalzed_doc_vectors_matrix = csr_matrix(np.vstack(normalzed_doc_vectors_by_group.values()))
    bow_with_concepts = csr_vappend(bow, normalzed_doc_vectors_matrix)
    doc_ids_with_concepts = doc_ids + normalzed_doc_vectors_by_group.keys()

    scores.score_variables(doc_ids_with_concepts, 
                           bow_with_concepts, 
                           num_cpus=num_cpus, 
                           file_name=out_file, 
                           pair_ids=pair_ids)
    
    
    return top_words_in_group

def calc_top_n_words_by_doc_cols(data_file, 
                                 doc_cols_inputs, 
                                 ref_id_col, 
                                 filter_file, 
                                 n, 
                                 concept_id_col, 
                                 filter_ref_id_col, 
                                 corpora_col=None, 
                                 word_vectors=None, 
                                 pair_ids=None):
    return {key: calc_top_n_words(data_file, 
                        doc_cols_inputs[key], 
                        key, 
                        ref_id_col, 
                        filter_file, 
                        n,
                        concept_id_col,
                        filter_ref_id_col, 
                        corpora_col=corpora_col, 
                        word_vectors=word_vectors, 
                        pair_ids=pair_ids)
                for key in doc_cols_inputs}

In [108]:
n = 5
data = pd.read_csv(data_file, sep=",",quotechar='"', na_values="",low_memory=False)
pair_ids = list(data[data["dbGaP_study_id"]=="SNOMED"][ref_id_col])
top_n_words = calc_top_n_words_by_doc_cols(data_file, 
                                            doc_cols_inputs, 
                                            ref_id_col, 
                                            man_file, 
                                            n,
                                            concept_id_col,
                                            filter_ref_id_col, 
                                            word_vectors=None, 
                                            pair_ids=pair_ids)
top_n_words

/Laura/automatic-variable-mapping/SNOMED-concepts/output/var_doc_obs_heart_studies_dbGaP_NLP_SNOMED_terms.csv
doc_col_key: desc
 Building Corpora...


100%|██████████| 133557/133557 [00:22<00:00, 5952.68it/s] 


('Corpus: ', 1, ' ', 133557)
  Calculating BOW matrix...
('vocab size: ', 42253)
  Calculating Top N words..
  Calculating Similarity Scores...
Finding valid pair ids
Pair ids: 27946
Finding valid ref ids
Ref ids: 22
Multiplying matrices
LHS: (27946, 42253)
RHS: (22, 42253)
Sim Matrix: (22, 27946)
Chunk size: 2
Finding matches for 0 community_5


100%|██████████| 22/22 [00:00<00:00, 13150.16it/s]


{'desc': {'community_1': ['id', 'subject', 'exam', 'aric', 'atrial'],
  'community_10': ['ldl', 'dl', 'cholesterol', 'mg', 'calibrated'],
  'community_11': ['height', 'index', 'mass', 'body', 'cm'],
  'community_12': ['chd', 'coronary', 'disease', 'heart', 'fatal'],
  'community_13': ['cigarette', 'smoked', 'smoking', 'year', 'last'],
  'community_14': ['rate', 'atrial', 'ecg', 'fibrillation', 'heart'],
  'community_15': ['cholesterol', 'mg', 'total', 'analysis', 'blood'],
  'community_16': ['id', 'participant', 'unique', 'share', 'number'],
  'community_17': ['zero', 'ave', 'hg', 'muddler', 'mm'],
  'community_18': ['marital', 'status', 'martial', '26a', 'aai'],
  'community_19': ['identifier', 'cohort', 'framingham', 'study', 'heart'],
  'community_2': ['age', 'exam', 'race', 'alcohol', 'interval'],
  'community_20': ['gender', 'hdl', 'cholesterol', 'dl', 'mg'],
  'community_21': ['related', 'procedure'],
  'community_22': ['activity', '3'],
  'community_3': ['diabetes', 'status', 'm

In [112]:
doc_col_key = "desc"  
filter_file = man_file
corpora_col = None
key = "desc"
doc_cols = doc_cols_inputs[key]
pair_ids_col = ref_id_col